BLG 453E - Computer Vision - Homework 1<br>Muhammed Tolga Cangöz - 150130024

In [1]:
from tqdm import tqdm
import numpy as np
import os
import cv2
import moviepy.editor as moviepy
import matplotlib.pyplot as plt
from PIL import Image as Img

pygame 2.0.3 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
planes = np.zeros((9,472,4,3))

for i in range(1,10):
	with open("Plane_"+str(i)+".txt") as f:
		content = f.readlines()
		for line_id in range(len(content)):
			sel_line = content[line_id]
			sel_line = sel_line.replace(')\n', '').replace("(", '').split(")")

			for point_id in range(4):
				sel_point = sel_line[point_id].split(" ")

				planes[i-1,line_id,point_id,0] = float(sel_point[0])
				planes[i-1,line_id,point_id,1] = float(sel_point[1])
				planes[i-1,line_id,point_id,2] = float(sel_point[2])

PART 3

In [3]:
images_list = []
vivaldi = cv2.imread('vivaldi.jpg')
imS = cv2.resize(vivaldi, (572, 322))
imS = cv2.flip(imS, 1)
cat_in_the_middle = Img.open('cat-headphones.png')
cat_in_the_middle = cat_in_the_middle.resize((572,322))
background = cat_in_the_middle.convert("RGBA")
source_points = np.array([[0, 0], [0, 322], [572, 322], [572, 0]], dtype=np.float32)



for i in tqdm(range(472)):
	blank_images = []
	blank_images_status = []

	for j in range(0,9):
		blank_image = np.zeros((322,572,3), np.uint8)

		pts = planes[j,i,:,:].squeeze()[:,0:2].astype(np.int32)

		temp = np.copy(pts[3,:])
		pts[3, :] = pts[2,:]
		pts[2, :] = temp

		temp = np.copy(pts[0,:])
		pts[:3, ] = pts[1:, :]
		pts[3, :] = temp
		
		blank_image[:imS.shape[0], :imS.shape[1]] = imS
		M = np.array([[source_points[0][0], source_points[0][1], 1, 0, 0, 0, -source_points[0][0]*pts[0][0], -source_points[0][1]*pts[0][0]],
					  [source_points[1][0], source_points[1][1], 1, 0, 0, 0, -source_points[1][0]*pts[1][0], -source_points[1][1]*pts[1][0]],
					  [source_points[2][0], source_points[2][1], 1, 0, 0, 0, -source_points[2][0]*pts[2][0], -source_points[2][1]*pts[2][0]],
					  [source_points[3][0], source_points[3][1], 1, 0, 0, 0, -source_points[3][0]*pts[3][0], -source_points[3][1]*pts[3][0]],
					  [0, 0, 0, source_points[0][0], source_points[0][1], 1, -source_points[0][0]*pts[0][1], -source_points[0][1]*pts[0][1]],
					  [0, 0, 0, source_points[1][0], source_points[1][1], 1, -source_points[1][0]*pts[1][1], -source_points[1][1]*pts[1][1]],
					  [0, 0, 0, source_points[2][0], source_points[2][1], 1, -source_points[2][0]*pts[2][1], -source_points[2][1]*pts[2][1]],
					  [0, 0, 0, source_points[3][0], source_points[3][1], 1, -source_points[3][0]*pts[3][1], -source_points[3][1]*pts[3][1]]])
		Inverse_M = np.linalg.pinv(M)
		points = np.array([pts[0][0], pts[1][0], pts[2][0], pts[3][0], pts[0][1], pts[1][1], pts[2][1], pts[3][1]])
		coefficients = np.dot(Inverse_M, points)
		transformationMatrix = np.concatenate([coefficients, np.array([1], dtype=np.float32)]).reshape((3,3))
		blank_image = cv2.warpPerspective(blank_image, transformationMatrix, (int(572), int(322)), flags=cv2.INTER_LINEAR)
		blank_images.append(Img.fromarray(cv2.cvtColor(blank_image, cv2.COLOR_BGR2RGB)))

		if pts[2,0] < pts[1,0]:
			blank_images_status.append(True)
		else:
			blank_images_status.append(False)

	final_image = Img.new("RGBA", background.size, "WHITE")
	first_time = True

	for a in sorted(zip(blank_images, blank_images_status), key=lambda x: x[1]):
		if not a[1]:
			open_cv_image = np.array(a[0])
			open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_RGB2BGR)
			tmp = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
			_,alpha = cv2.threshold(tmp,0,255, cv2.THRESH_BINARY)
			b, g, r = cv2.split(open_cv_image)
			rgba = [b,g,r, alpha]
			dst = cv2.merge(rgba,4)
			final_image.paste(Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)), (0,0), Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)))
		
		else:
			if first_time:
				final_image.paste(background, (0,0), background)	# Adding cat in the middle
				first_time = False
			open_cv_image = np.array(a[0])
			open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_RGB2BGR)
			tmp = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
			_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
			b, g, r = cv2.split(open_cv_image)
			rgba = [b,g,r, alpha]
			dst = cv2.merge(rgba,4)
			final_image.paste(Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)), (0,0), Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)))
	
	images_list.append(np.array(final_image.convert('RGB')))


clip = moviepy.ImageSequenceClip(images_list, fps = 25)
clip.write_videofile("part3.mp4", codec="libx264")

100%|██████████| 472/472 [00:21<00:00, 21.81it/s]


Moviepy - Building video part3.mp4.
Moviepy - Writing video part3.mp4



Moviepy - Done !
Moviepy - video ready part3.mp4


PART 4-A

In [6]:
vivaldi = cv2.imread('vivaldi.jpg')
image = cv2.flip(vivaldi, 1)
theta = np.pi/3
centerX = image.shape[1]/2
centerY = image.shape[0]/2

rotationMatrix = np.array([[np.cos(theta),  np.sin(theta), centerX*(1 - np.cos(theta)) - centerY*np.sin(theta)],
                           [-np.sin(theta), np.cos(theta), centerY*(1 - np.cos(theta)) + centerX*np.sin(theta)]])

# Arrange new coordinates so that new shape can be seen totally without cutting
new_witdh = int((image.shape[0] * np.abs(rotationMatrix[0, 1])) + (image.shape[1] * np.abs(rotationMatrix[0, 0])))
new_hight = int((image.shape[0] * np.abs(rotationMatrix[0, 0])) + (image.shape[1] * np.abs(rotationMatrix[0, 1])))

# Update rotation points
rotationMatrix[0, 2] += (new_witdh / 2) - centerX
rotationMatrix[1, 2] += (new_hight / 2) - centerY

imS =  cv2.warpAffine(image, rotationMatrix, (new_witdh, new_hight))


images_list = []
imS = cv2.resize(imS, (572, 322))
cat_in_the_middle = Img.open('cat-headphones.png')
cat_in_the_middle = cat_in_the_middle.resize((572,322))
background = cat_in_the_middle.convert("RGBA")
source_points = np.array([[0, 0], [0, 322], [572, 322], [572, 0]], dtype=np.float32)


for i in tqdm(range(472)):
	blank_images = []
	blank_images_status = []

	for j in range(0,9):
		blank_image = np.zeros((322,572,3), np.uint8)

		pts = planes[j,i,:,:].squeeze()[:,0:2].astype(np.int32)

		temp = np.copy(pts[3,:])
		pts[3, :] = pts[2,:]
		pts[2, :] = temp

		temp = np.copy(pts[0,:])
		pts[:3, ] = pts[1:, :]
		pts[3, :] = temp

		blank_image[:imS.shape[0], :imS.shape[1]] = imS
		M = np.array([[source_points[0][0], source_points[0][1], 1, 0, 0, 0, -source_points[0][0]*pts[0][0], -source_points[0][1]*pts[0][0]],
					  [source_points[1][0], source_points[1][1], 1, 0, 0, 0, -source_points[1][0]*pts[1][0], -source_points[1][1]*pts[1][0]],
					  [source_points[2][0], source_points[2][1], 1, 0, 0, 0, -source_points[2][0]*pts[2][0], -source_points[2][1]*pts[2][0]],
					  [source_points[3][0], source_points[3][1], 1, 0, 0, 0, -source_points[3][0]*pts[3][0], -source_points[3][1]*pts[3][0]],
					  [0, 0, 0, source_points[0][0], source_points[0][1], 1, -source_points[0][0]*pts[0][1], -source_points[0][1]*pts[0][1]],
					  [0, 0, 0, source_points[1][0], source_points[1][1], 1, -source_points[1][0]*pts[1][1], -source_points[1][1]*pts[1][1]],
					  [0, 0, 0, source_points[2][0], source_points[2][1], 1, -source_points[2][0]*pts[2][1], -source_points[2][1]*pts[2][1]],
					  [0, 0, 0, source_points[3][0], source_points[3][1], 1, -source_points[3][0]*pts[3][1], -source_points[3][1]*pts[3][1]]])
		Inverse_M = np.linalg.pinv(M)
		points = np.array([pts[0][0], pts[1][0], pts[2][0], pts[3][0], pts[0][1], pts[1][1], pts[2][1], pts[3][1]])
		coefficients = np.dot(Inverse_M, points)
		transformationMatrix = np.concatenate([coefficients, np.array([1], dtype=np.float32)]).reshape((3,3))
		blank_image = cv2.warpPerspective(blank_image, transformationMatrix, (int(572), int(322)), flags=cv2.INTER_LINEAR)
		blank_images.append(Img.fromarray(cv2.cvtColor(blank_image, cv2.COLOR_BGR2RGB)))

		if pts[2,0] < pts[1,0]:
			blank_images_status.append(True)
		else:
			blank_images_status.append(False)

	final_image = Img.new("RGBA", background.size, "WHITE")
	first_time = True

	for a in sorted(zip(blank_images, blank_images_status), key=lambda x: x[1]):
		if not a[1]:
			open_cv_image = np.array(a[0])
			open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_RGB2BGR)
			tmp = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
			_,alpha = cv2.threshold(tmp,0,255, cv2.THRESH_BINARY)
			b, g, r = cv2.split(open_cv_image)
			rgba = [b,g,r, alpha]
			dst = cv2.merge(rgba,4)
			final_image.paste(Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)), (0,0), Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)))
		
		else:
			if first_time:
				final_image.paste(background, (0,0), background)
				first_time = False
			open_cv_image = np.array(a[0])
			open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_RGB2BGR)
			tmp = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
			_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
			b, g, r = cv2.split(open_cv_image)
			rgba = [b,g,r, alpha]
			dst = cv2.merge(rgba,4)
			final_image.paste(Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)), (0,0), Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)))

	images_list.append(np.array(final_image.convert('RGB')))


clip = moviepy.ImageSequenceClip(images_list, fps = 25)
clip.write_videofile("part4_60_degree_from_center.mp4", codec="libx264")

100%|██████████| 472/472 [00:20<00:00, 22.61it/s]


Moviepy - Building video part4_60_degree_from_center.mp4.
Moviepy - Writing video part4_60_degree_from_center.mp4



Moviepy - Done !
Moviepy - video ready part4_60_degree_from_center.mp4


PART 4-B

In [7]:
theta = np.pi/3
rotationMatrix = np.array([[np.cos(theta),  np.sin(theta), 0],
                           [-np.sin(theta), np.cos(theta), 0]])

vivaldi = cv2.imread('vivaldi.jpg')
image = cv2.flip(vivaldi, 1)
w = int(image.shape[0] * abs(rotationMatrix[0, 1]) + image.shape[1] * abs(rotationMatrix[0, 0]))
h = int(image.shape[0] * abs(rotationMatrix[0, 0]) + image.shape[1] * abs(rotationMatrix[0, 1]))
rotationMatrix += np.asarray([[0, 0, +0], [0, 0, +1050]])  # just locating to a visible area
imS = cv2.warpAffine(image, rotationMatrix, (w, h), borderMode=cv2.BORDER_CONSTANT)

images_list = []
imS = cv2.resize(imS, (572, 322))
cat_in_the_middle = Img.open('cat-headphones.png')
cat_in_the_middle = cat_in_the_middle.resize((572,322))
background = cat_in_the_middle.convert("RGBA")
source_points = np.array([[0, 0], [0, 322], [572, 322], [572, 0]], dtype=np.float32)


for i in tqdm(range(472)):
	blank_images = []
	blank_images_status = []

	for j in range(0,9):
		blank_image = np.zeros((322,572,3), np.uint8)

		pts = planes[j,i,:,:].squeeze()[:,0:2].astype(np.int32)

		temp = np.copy(pts[3,:])
		pts[3, :] = pts[2,:]
		pts[2, :] = temp

		temp = np.copy(pts[0,:])
		pts[:3, ] = pts[1:, :]
		pts[3, :] = temp

		blank_image[:imS.shape[0], :imS.shape[1]] = imS
		M = np.array([[source_points[0][0], source_points[0][1], 1, 0, 0, 0, -source_points[0][0]*pts[0][0], -source_points[0][1]*pts[0][0]],
					  [source_points[1][0], source_points[1][1], 1, 0, 0, 0, -source_points[1][0]*pts[1][0], -source_points[1][1]*pts[1][0]],
					  [source_points[2][0], source_points[2][1], 1, 0, 0, 0, -source_points[2][0]*pts[2][0], -source_points[2][1]*pts[2][0]],
					  [source_points[3][0], source_points[3][1], 1, 0, 0, 0, -source_points[3][0]*pts[3][0], -source_points[3][1]*pts[3][0]],
					  [0, 0, 0, source_points[0][0], source_points[0][1], 1, -source_points[0][0]*pts[0][1], -source_points[0][1]*pts[0][1]],
					  [0, 0, 0, source_points[1][0], source_points[1][1], 1, -source_points[1][0]*pts[1][1], -source_points[1][1]*pts[1][1]],
					  [0, 0, 0, source_points[2][0], source_points[2][1], 1, -source_points[2][0]*pts[2][1], -source_points[2][1]*pts[2][1]],
					  [0, 0, 0, source_points[3][0], source_points[3][1], 1, -source_points[3][0]*pts[3][1], -source_points[3][1]*pts[3][1]]])
		Inverse_M = np.linalg.pinv(M)
		points = np.array([pts[0][0], pts[1][0], pts[2][0], pts[3][0], pts[0][1], pts[1][1], pts[2][1], pts[3][1]])
		coefficients = np.dot(Inverse_M, points)
		transformationMatrix = np.concatenate([coefficients, np.array([1], dtype=np.float32)]).reshape((3,3))
		blank_image = cv2.warpPerspective(blank_image, transformationMatrix, (int(572), int(322)), flags=cv2.INTER_LINEAR)
		blank_images.append(Img.fromarray(cv2.cvtColor(blank_image, cv2.COLOR_BGR2RGB)))

		if pts[2,0] < pts[1,0]:
			blank_images_status.append(True)
		else:
			blank_images_status.append(False)

	final_image = Img.new("RGBA", background.size, "WHITE")
	first_time = True

	for a in sorted(zip(blank_images, blank_images_status), key=lambda x: x[1]):
		if not a[1]:
			open_cv_image = np.array(a[0])
			open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_RGB2BGR)
			tmp = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
			_,alpha = cv2.threshold(tmp,0,255, cv2.THRESH_BINARY)
			b, g, r = cv2.split(open_cv_image)
			rgba = [b,g,r, alpha]
			dst = cv2.merge(rgba,4)
			final_image.paste(Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)), (0,0), Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)))
		
		else:
			if first_time:
				final_image.paste(background, (0,0), background)
				first_time = False
			open_cv_image = np.array(a[0])
			open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_RGB2BGR)
			tmp = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
			_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
			b, g, r = cv2.split(open_cv_image)
			rgba = [b,g,r, alpha]
			dst = cv2.merge(rgba,4)
			final_image.paste(Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)), (0,0), Img.fromarray(cv2.cvtColor(dst, cv2.COLOR_BGRA2RGBA)))

	images_list.append(np.array(final_image.convert('RGB')))


clip = moviepy.ImageSequenceClip(images_list, fps = 25)
clip.write_videofile("part4_60_degree_from_x=0_y=0.mp4", codec="libx264")

100%|██████████| 472/472 [00:26<00:00, 17.96it/s]


Moviepy - Building video part4_60_degree_from_x=0_y=0.mp4.
Moviepy - Writing video part4_60_degree_from_x=0_y=0.mp4



Moviepy - Done !
Moviepy - video ready part4_60_degree_from_x=0_y=0.mp4


They are the same.